# MOUNT DRIVE !!!

In [10]:
# Hyperparameters
device = "cuda"
split = "validation"
mbs = 256

In [2]:
!pip install datasets
!pip install ctranslate2
!pip install OpenNMT-py=2.*
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.7/36.7 MB 30.4 MB/s eta 0:00:00
ERROR: Invalid requirement: 'OpenNMT-py=2.*'
Hint: = is not a valid operator. Did you mean == ?


In [3]:
import ctranslate2
import sentencepiece as spm
from datasets import load_dataset
from tqdm.auto import tqdm
import os
import torch

In [11]:
# get data
os.makedirs("/content/drive/My Drive/TinyStoriesTranslate/", exist_ok=True)
dataset = load_dataset("roneneldan/TinyStories", split=split)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


In [5]:
# get model
!wget https://object.pouta.csc.fi/OPUS-MT-models/en-de/opus-2020-02-26.zip
!unzip opus-2020-02-26.zip
!ct2-opus-mt-converter --model_dir . --output_dir ende_ctranslate2

--2024-03-17 17:29:50--  https://object.pouta.csc.fi/OPUS-MT-models/en-de/opus-2020-02-26.zip
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 275458389 (263M) [application/zip]
Saving to: ‘opus-2020-02-26.zip’

opus-2020-02-26.zip 100%[===================>] 262.70M  17.5MB/s    in 26s     

2024-03-17 17:30:17 (10.0 MB/s) - ‘opus-2020-02-26.zip’ saved [275458389/275458389]

Archive:  opus-2020-02-26.zip
  inflating: README.md               
  inflating: LICENSE                 
  inflating: opus.spm32k-spm32k.transformer-align.model1.npz.best-perplexity.npz  
  inflating: opus.spm32k-spm32k.vocab.yml  
  inflating: opus.spm32k-spm32k.transformer-align.valid1.log  
  inflating: opus.spm32k-spm32k.transformer-align.train1.log  
  inflating: source.spm              
 extracting: source.tcmodel          
  inflating

In [6]:
sp = spm.SentencePieceProcessor()
sp.load("source.spm")

translator = ctranslate2.Translator(
    "ende_ctranslate2",
    compute_type="auto",
    device=device,
)
print(f"Using compute type: {translator.compute_type}")


Using compute type: int8_float16


In [7]:
def translate(start_ind, end_ind, verbose=True, mbs=256, split="train"):
    if verbose: print("Encoding...")
    encodings = []
    for story in dataset[start_ind:end_ind]["text"]:
      encodings.append(sp.encode(story, out_type=str))
    if verbose: print("Translating...")
    translations_encoded = translator.translate_batch(encodings, beam_size=1, max_batch_size=mbs)
    if verbose: print("Decoding...")
    translations = [sp.decode(tra.hypotheses[0]) for tra in translations_encoded]
    if verbose: print("Saving...")
    with open(f"/content/drive/My Drive/TinyStoriesTranslate/TS_{split}_de_{start_ind}_{end_ind}.csv", "w") as f:
        for story in translations:
            story = story.replace("▁", " ")
            f.write(story + "\n")
    if verbose: print("Done")

In [13]:
for i in tqdm(range(0, len(dataset)+1, 10000)):
    translate(i, i+10000, mbs=mbs, split=split, verbose=False)

  0%|          | 0/3 [00:00<?, ?it/s]

In [14]:
print(len(dataset))

21990
